Step 1: Collect raw data.

In [1]:
import urllib.request
import datetime
import os

In [2]:
for i in range(1,28):
    url="https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={}&year1=1981&year2=2024&type=Mean".format(i)
    wp = urllib.request.urlopen(url)
    text = wp.read()
    now = datetime.datetime.now()
    date_and_time_time = now.strftime("%d%m%Y%H%M%S")
    out = open(os.path.join('raw', 'NOAA_ID'+str(i)+'_'+date_and_time_time+'.csv'), 'wb')
    out.write(text)
    out.close()

Step 2: Creat dataframe.

In [3]:
import pandas as pd

In [4]:
directory = 'raw'
files = files = sorted(os.listdir(directory), key=lambda x: int(x.split('_')[-1].split('.')[0]))
uncombined_df = []
headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'Area']
i = 1

index_changer = {
    1:'Черкаська', 2:'Чернігівська', 3:'Чернівецька', 4:'Республіка Крим', 5:'Дніпропетровська',
    6:'Донецька', 7:'Івано-Франківська', 8:'Харківська', 9:'Херсонська', 10:'Хмельницька',
    11:'Київська', 12:'Київ', 13:'Кіровоградська', 14:'Луганська', 15:'Львівська',
    16:'Миколаївська', 17:'Одеська', 18:'Полтавська', 19:'Рівеньска', 20:'Севастопіль',
    21:'Сумьска', 22:'Тернопільська', 23:'Закарпатська', 24:'Вінницька', 25:'Волинська',
    26:'Запорізька', 27:'Житомирська',
}

In [5]:
for file in files:
    file_path = os.path.join(directory, file)
    dataframe = pd.read_csv(file_path, header = 1, names = headers)
    dataframe.drop(dataframe.index[-1], inplace=True)
    dataframe['Area'] = i
    i += 1

    dataframe = dataframe.drop(dataframe.loc[dataframe['VHI'] == -1].index)
    dataframe['Area'] = dataframe['Area'].replace({index: area for index, area in index_changer.items()})

    uncombined_df.append(dataframe)

df = pd.concat(uncombined_df, ignore_index=True)

Step 3: Procedures implementation

In [6]:
def VHI_of_year_finder(area, year):
    finded = df[(df["Area"] == area) & (df["Year"] == str(year))]
    for index, row in finded.iterrows():
        print(f"Тиждень: {row['Week']}, VHI: {row['VHI']}")

    min_v = df[(df['Year'].astype(str) == str(year)) & (df["Area"] == area)]['VHI'].min()
    print('Найменший показник VHI у', area, 'за', year, 'рік: ', min_v)
    max_v = df[(df['Year'].astype(str) == str(year)) & (df["Area"] == area)]['VHI'].max()
    print('Найбільший показник VHI у', area, 'за', year, 'рік: ', max_v)

In [7]:
VHI_of_year_finder('Черкаська', 1987)

Тиждень: 1.0, VHI: 44.86
Тиждень: 2.0, VHI: 46.66
Тиждень: 3.0, VHI: 48.18
Тиждень: 4.0, VHI: 46.91
Тиждень: 5.0, VHI: 45.38
Тиждень: 6.0, VHI: 45.14
Тиждень: 7.0, VHI: 43.92
Тиждень: 8.0, VHI: 44.53
Тиждень: 9.0, VHI: 45.65
Тиждень: 10.0, VHI: 47.62
Тиждень: 11.0, VHI: 48.04
Тиждень: 12.0, VHI: 47.25
Тиждень: 13.0, VHI: 46.06
Тиждень: 14.0, VHI: 45.66
Тиждень: 15.0, VHI: 45.84
Тиждень: 16.0, VHI: 45.25
Тиждень: 17.0, VHI: 41.42
Тиждень: 18.0, VHI: 38.13
Тиждень: 19.0, VHI: 38.22
Тиждень: 20.0, VHI: 35.6
Тиждень: 21.0, VHI: 34.61
Тиждень: 22.0, VHI: 36.14
Тиждень: 23.0, VHI: 41.03
Тиждень: 24.0, VHI: 46.38
Тиждень: 25.0, VHI: 51.75
Тиждень: 26.0, VHI: 55.46
Тиждень: 27.0, VHI: 56.73
Тиждень: 28.0, VHI: 57.28
Тиждень: 29.0, VHI: 55.94
Тиждень: 30.0, VHI: 55.47
Тиждень: 31.0, VHI: 57.02
Тиждень: 32.0, VHI: 58.17
Тиждень: 33.0, VHI: 58.66
Тиждень: 34.0, VHI: 58.6
Тиждень: 35.0, VHI: 57.27
Тиждень: 36.0, VHI: 56.16
Тиждень: 37.0, VHI: 56.04
Тиждень: 38.0, VHI: 55.43
Тиждень: 39.0, VHI: 53.

In [8]:
def VHI_of_range_finder(areas, start_year, end_year):
    for year in range(start_year, end_year + 1):
        print(f"Рік: {year}")
        for area in areas:
            filtered_data = df[(df['Year'] == str(year)) & (df['Area'] == area)]
            print(f"Область: {area}")
            for index, row in filtered_data.iterrows():
                print(f"Тиждень: {row['Week']}, VHI: {row['VHI']}")
            print()
        print()

In [9]:
VHI_of_range_finder(['Черкаська', 'Запорізька'], 1987, 1989)

Рік: 1987
Область: Черкаська
Тиждень: 1.0, VHI: 44.86
Тиждень: 2.0, VHI: 46.66
Тиждень: 3.0, VHI: 48.18
Тиждень: 4.0, VHI: 46.91
Тиждень: 5.0, VHI: 45.38
Тиждень: 6.0, VHI: 45.14
Тиждень: 7.0, VHI: 43.92
Тиждень: 8.0, VHI: 44.53
Тиждень: 9.0, VHI: 45.65
Тиждень: 10.0, VHI: 47.62
Тиждень: 11.0, VHI: 48.04
Тиждень: 12.0, VHI: 47.25
Тиждень: 13.0, VHI: 46.06
Тиждень: 14.0, VHI: 45.66
Тиждень: 15.0, VHI: 45.84
Тиждень: 16.0, VHI: 45.25
Тиждень: 17.0, VHI: 41.42
Тиждень: 18.0, VHI: 38.13
Тиждень: 19.0, VHI: 38.22
Тиждень: 20.0, VHI: 35.6
Тиждень: 21.0, VHI: 34.61
Тиждень: 22.0, VHI: 36.14
Тиждень: 23.0, VHI: 41.03
Тиждень: 24.0, VHI: 46.38
Тиждень: 25.0, VHI: 51.75
Тиждень: 26.0, VHI: 55.46
Тиждень: 27.0, VHI: 56.73
Тиждень: 28.0, VHI: 57.28
Тиждень: 29.0, VHI: 55.94
Тиждень: 30.0, VHI: 55.47
Тиждень: 31.0, VHI: 57.02
Тиждень: 32.0, VHI: 58.17
Тиждень: 33.0, VHI: 58.66
Тиждень: 34.0, VHI: 58.6
Тиждень: 35.0, VHI: 57.27
Тиждень: 36.0, VHI: 56.16
Тиждень: 37.0, VHI: 56.04
Тиждень: 38.0, VHI: 

In [10]:
def drought_finder(percentage):
    for year in df['Year'].unique():
        drought_counter = 0
        slight_drought_counter = 0
        for area in df['Area'].unique():
            area_data = df[(df['Year'] == year) & (df['Area'] == area)]
            if (area_data['VHI'] < 15).any():
                drought_counter += 1
            elif (area_data['VHI'] < 35).any():
                slight_drought_counter += 1
        if drought_counter/27 * 100 > percentage:
            print(f'Рік, коли Екстримальні посухи торкнулися більше {percentage}% областей по Україні: ', year)
        if slight_drought_counter/27 * 100 > percentage:
            print(f'Рік, коли Помірні посухи торкнулися більше {percentage}% областей по Україні: ', year)

In [11]:
drought_finder(20)

Рік, коли Помірні посухи торкнулися більше 20% областей по Україні:  1982
Рік, коли Помірні посухи торкнулися більше 20% областей по Україні:  1983
Рік, коли Помірні посухи торкнулися більше 20% областей по Україні:  1984
Рік, коли Помірні посухи торкнулися більше 20% областей по Україні:  1985
Рік, коли Помірні посухи торкнулися більше 20% областей по Україні:  1986
Рік, коли Помірні посухи торкнулися більше 20% областей по Україні:  1987
Рік, коли Помірні посухи торкнулися більше 20% областей по Україні:  1988
Рік, коли Помірні посухи торкнулися більше 20% областей по Україні:  1989
Рік, коли Помірні посухи торкнулися більше 20% областей по Україні:  1990
Рік, коли Помірні посухи торкнулися більше 20% областей по Україні:  1991
Рік, коли Помірні посухи торкнулися більше 20% областей по Україні:  1992
Рік, коли Помірні посухи торкнулися більше 20% областей по Україні:  1993
Рік, коли Помірні посухи торкнулися більше 20% областей по Україні:  1994
Рік, коли Помірні посухи торкнулися бі